# Cotação de FIIs

Script que extrai a cotação atualizada de cotas de fundos imobiliários escolhidas para um arquivo .csv.

In [ ]:
# Instalação das bibliotecas
#!pip install requests --quiet
#!pip install pandas --quiet
#!pip install beautifulsoup4 --quiet
#!pip install matplotlib

In [1]:
# Importando as bibliotecas
import datetime
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# URL com a tabela de dados
url = "https://www.fundamentus.com.br/fii_resultado.php"

# Obtendo o conteúdo da página em formato de texto

# https://stackoverflow.com/questions/68259148/getting-404-error-for-certain-stocks-and-pages-on-yahoo-finance-python
headers = { 
    'User-Agent'      : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36', 
    'Accept'          : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 
    'Accept-Language' : 'en-US,en;q=0.5',
    'DNT'             : '1', # Do Not Track Request Header 
    'Connection'      : 'close'
}
data = requests.get(url, headers=headers, timeout=5).text
soup = BeautifulSoup(data,"html.parser")

# Procurando a tabela da página
table = soup.find('table') # em html uma tabela é representada pela tag <table>

# Definindo dataframe
df = pd.DataFrame(columns=['Papel', 'Tipo', 'Segmento', 'Administradora', 'Cotação', 'DY (12M)', 'DY Atual', 'PVP',
                 'Liquidez diária', 'Valor Patrimonial', 'Valor de mercado','Vacância média',
                 'Último dividendo', 'Magic Number', 'Valor do Magic Number', 'FFO Yield', 
                 'Cap Rate', 'Qtd de imóveis', 'Preço do m²', 'Aluguel por m²'])

# Obtendo todas as linhas da tabela
for row in table.tbody.find_all('tr'): # em html uma linha da tabela é representada pela tag <tr>
    # Obtendo todas as colunas em cada linha
    columns = row.find_all('td')  # em html uma coluna da tabela é representada pela tag <td>
    if(columns != []):
        papel = columns[0].text.strip(' ')
        segmento = columns[1].text.strip(' ')
        cotacao = columns[2].text.strip(' ')
        ffo = columns[3].text.strip(' ')
        dy12 = columns[4].text.strip(' ')
        pvp = columns[5].text.strip(' ')
        vlrmercado = columns[6].text.strip(' ')
        liqdiaria = columns[7].text.strip(' ')
        qtdimoveis = columns[8].text.strip(' ')
        precom2 = columns[9].text.strip(' ')
        aluguelm2 = columns[10].text.strip(' ')
        caprate = columns[11].text.strip(' ')
        vacancia = columns[12].text.strip(' ')
        df = pd.concat([df, pd.DataFrame.from_records([{'Papel': papel,  'Segmento': segmento, 'Cotação': cotacao,
                                                        'FFO Yield': ffo, 'DY (12M)': dy12, 'PVP':pvp, 'Valor de mercado': vlrmercado,
                                                        'Liquidez diária': liqdiaria, 'Qtd de imóveis': qtdimoveis,
                                                        'Preço do m²': precom2, 'Aluguel por m²': aluguelm2,
                                                        'Cap Rate': caprate, 'Vacância média': vacancia}])], ignore_index=True)

listaFiltro = ['BRCR11', 'BTCI11', 'BTLG11', 'CPTS11', 'GARE11', 'HGBS11', 'HGLG11', 'HGRE11', 'HGRU11', 'JSRE11', 'KNCA11',
               'KNCR11', 'KNRI11', 'KNSC11', 'MALL11', 'MXRF11', 'SNAG11', 'URPR11', 'VGHF11', 'VGIA11', 'VINO11', 'VISC11',
               'XPCA11', 'XPLG11', 'XPML11']

df = df[df.Papel.isin(listaFiltro)]
df = df.reset_index(drop=True)

for item in listaFiltro:
    url="https://www.fundamentus.com.br/fii_proventos.php?papel="+item+"&tipo=2"
    data = requests.get(url, headers=headers, timeout=5).text
    soup = BeautifulSoup(data,"html.parser")
    table = soup.find('table') # em html uma tabela é representada pela tag <table>
    df2 = pd.DataFrame(columns=['Último dividendo'])
    for row in table.tbody.find_all('tr'): # em html uma linha da tabela é representada pela tag <tr>
        columns = row.find_all('td') # em html uma coluna da tabela é representada pela tag <td>
        if(columns != []):
            dividendo = columns[3].text.strip(' ')
            df2 = pd.concat([df2, pd.DataFrame.from_records([{'Último dividendo': dividendo}])], ignore_index=True)
    df['Último dividendo'].iloc[listaFiltro.index(item)]=df2.iloc[0,0]

for item in listaFiltro:
    url="https://investidor10.com.br/fiis/"+item
    data = requests.get(url, headers=headers, timeout=5).text
    soup = BeautifulSoup(data,"html.parser")
    adm = soup.find('h2', class_="name-company").get_text()
    tipo = soup.findAll(True, {'class':['value']})[6].get_text()[34:-30]
    segmento = soup.findAll(True, {'class':['value']})[5].get_text()[34:-30]
    df['Administradora'].iloc[listaFiltro.index(item)] = adm
    df['Tipo'].iloc[listaFiltro.index(item)] = tipo
    df['Segmento'].iloc[listaFiltro.index(item)] = segmento
    
for item in listaFiltro:
    url="https://www.fundamentus.com.br/detalhes.php?papel="+item
    data = requests.get(url, headers=headers, timeout=5).text
    soup = BeautifulSoup(data,"html.parser")
    vlrpatrimonial = soup.findAll(True, {'class':['txt']})[75].get_text()
    df['Valor Patrimonial'].iloc[listaFiltro.index(item)] = vlrpatrimonial    

df['Cotação'] = df['Cotação'].replace({',': '.'}, regex=True)
df['Último dividendo'] = df['Último dividendo'].replace({',': '.'}, regex=True)
df['Cotação'] = df['Cotação'].astype(float)
df['Último dividendo'] = df['Último dividendo'].astype(float)
df['Magic Number'] = (df['Cotação'] / df['Último dividendo']).apply(np.ceil) 
df['Valor do Magic Number'] = (df['Magic Number'] * df['Cotação']).round(2)
df['DY Atual'] = ((((df['Último dividendo'] / df['Cotação'] + 1)**12) -1)* 100).round(2)

df['Cotação'] = df['Cotação'].astype(str)
df['Último dividendo'] = df['Último dividendo'].astype(str)
df['Valor do Magic Number'] = df['Valor do Magic Number'].astype(str)
df['DY Atual'] = df['DY Atual'].astype(str)
df['Magic Number'] = df['Magic Number'].astype(int)

df['Cotação'] = df['Cotação'].str.replace('.', ',')
df['Último dividendo'] = df['Último dividendo'].str.replace('.', ',')
df['Valor do Magic Number'] = df['Valor do Magic Number'].str.replace('.', ',')
df['DY Atual'] = df['DY Atual'].str.replace('.', ',')+"%"

df.to_csv(r'C:/1/FII.csv', index = False, encoding = 'utf-8-sig')
print("Concluído às " + datetime.datetime.now().strftime("%H:%M:%S de %d/%m/%Y"))
#df

Concluído às 14:13:33 de 03/02/2025
